In [1]:
import pandas as pd

In [2]:
netzero = pd.read_csv(filepath_or_buffer="/Users/christiannielsen/Library/CloudStorage/OneDrive-Personal/Dokumenter/AAU Datavidenskab/6. Semester/P6 Project/Net zero house/All-Subsystems-minute.csv")

In [3]:
netzero

,Timestamp,TimeStamp_Count,DHW_ClothesWasherColdFlow,DHW_ClothesWasherHotFlow,DHW_DishwasherHotFlow,DHW_HeatPumpWaterHeaterEnergyTotal,DHW_HeatPumpWaterHeaterPowerTotal,DHW_ManifoldColdFlow,DHW_ManifoldHotFlow,DHW_MixValveColdFlow,...,Vent_HRVTempReturnIn,Vent_HRVTempSupplyOut,Vent_HRVfreshairpressuredifferential,Vent_HRVstaleairpressuredifferential,HVAC_HeatPumpIndoorUnitPower,HVAC_HeatPumpOutdoorUnitPower,HVAC_DehumidifierAirflow,HVAC_DehumidifierExitAirTemp,HVAC_DehumidifierInletAirTemp,HVAC_DehumidifierPower
0,2013-07-01 00:01:14-04:00,1,0.0,0.0,0.00000,0.000000,6.920000,0.000000,0.00000,0.00000,...,22.552117,23.758415,NaN,NaN,125.4000,922.6568,NaN,NaN,NaN,NaN
1,2013-07-01 00:02:14-04:00,2,0.0,0.0,0.00000,0.132000,6.920000,0.000000,0.00000,0.00000,...,22.523656,23.737489,NaN,NaN,127.0400,919.2424,NaN,NaN,NaN,NaN
2,2013-07-01 00:03:14-04:00,3,0.0,0.0,0.00000,0.264000,6.920000,0.000000,0.00000,0.00000,...,22.539345,23.742217,NaN,NaN,153.3050,920.8184,NaN,NaN,NaN,NaN
3,2013-07-01 00:04:14-04:00,4,0.0,0.0,0.00000,0.396000,6.920000,0.000000,0.00000,0.00000,...,22.539554,23.727526,NaN,NaN,143.4575,922.3944,NaN,NaN,NaN,NaN
4,2013-07-01 00:05:14-04:00,5,0.0,0.0,0.00000,0.528000,6.920000,0.000000,0.00000,0.00000,...,22.506425,23.705258,NaN,NaN,141.8150,925.2832,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518787,2014-06-30 23:29:22-04:00,518788,0.0,0.0,1.86261,1074.687075,765.920563,11.297412,57.77676,8.69218,...,23.912489,24.423000,2.209914,2.503488,110.6250,1517.2800,NaN,NaN,NaN,NaN
518788,2014-06-30 23:30:22-04:00,518789,0.0,0.0,1.86261,1087.432650,766.929871,11.297412,57.77676,8.69218,...,23.898320,24.403229,2.195995,2.512385,124.5800,1394.6256,NaN,NaN,NaN,NaN
518789,2014-06-30 23:31:22-04:00,518790,0.0,0.0,1.86261,1100.200137,768.525940,11.297412,57.77676,8.69218,...,23.849174,24.392863,2.224423,2.554394,131.1450,1311.6304,NaN,NaN,NaN,NaN
518790,2014-06-30 23:59:22-04:00,518791,0.0,0.0,1.86261,1173.107485,6.920000,11.297412,57.77676,8.69218,...,24.119352,24.579783,2.239691,2.474923,127.0400,1325.5504,NaN,NaN,NaN,NaN


In [6]:
netzero.describe().T

,count,mean,std,min,25%,50%,75%,max
TimeStamp_Count,518792.0,259396.500000,149762.494764,1.0000,129698.750000,259396.500000,389094.250000,518792.000000
DHW_ClothesWasherColdFlow,518779.0,3.984146,8.999488,0.0000,0.000000,0.000000,0.000000,33.064630
DHW_ClothesWasherHotFlow,518779.0,0.741001,5.940131,0.0000,0.000000,0.026420,0.079260,201.696899
DHW_DishwasherHotFlow,518779.0,0.170642,0.523215,0.0000,0.000000,0.000000,0.000000,4.636710
DHW_HeatPumpWaterHeaterEnergyTotal,518792.0,1523.160343,4743.410245,0.0000,132.720673,1049.752184,2077.876207,188169.777407
...,...,...,...,...,...,...,...,...
HVAC_HeatPumpOutdoorUnitPower,515297.0,507.380077,618.862036,-68.7461,39.390400,41.228800,1137.236000,8624.912000
HVAC_DehumidifierAirflow,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HVAC_DehumidifierExitAirTemp,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HVAC_DehumidifierInletAirTemp,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
import json
EV = json.load(open('/Users/christiannielsen/Library/CloudStorage/OneDrive-Personal/Dokumenter/AAU Datavidenskab/6. Semester/P6 Project/EV scheduling/acndata_sessions.json'))

EV

{'_meta': {'end': 'Tue, 01 Feb 2022 07:04:00 GMT',
  'min_kWh': 0,
  'site': 'caltech',
  'start': 'Tue, 02 Jan 2018 07:04:00 GMT'},
 '_items': [{'_id': '5bc90cb9f9af8b0d7fe77cd2',
   'clusterID': '0039',
   'connectionTime': 'Wed, 25 Apr 2018 11:08:04 GMT',
   'disconnectTime': 'Wed, 25 Apr 2018 13:20:10 GMT',
   'doneChargingTime': 'Wed, 25 Apr 2018 13:21:10 GMT',
   'kWhDelivered': 7.932,
   'sessionID': '2_39_78_362_2018-04-25 11:08:04.400812',
   'siteID': '0002',
   'spaceID': 'CA-496',
   'stationID': '2-39-78-362',
   'timezone': 'America/Los_Angeles',
   'userID': None,
   'userInputs': None},
  {'_id': '5bc90cb9f9af8b0d7fe77cd3',
   'clusterID': '0039',
   'connectionTime': 'Wed, 25 Apr 2018 13:45:10 GMT',
   'disconnectTime': 'Thu, 26 Apr 2018 00:56:16 GMT',
   'doneChargingTime': 'Wed, 25 Apr 2018 16:44:15 GMT',
   'kWhDelivered': 10.013,
   'sessionID': '2_39_95_27_2018-04-25 13:45:09.617470',
   'siteID': '0002',
   'spaceID': 'CA-319',
   'stationID': '2-39-95-27',
   't

In [20]:
# EV = pd.read_json(path_or_buf="/Users/christiannielsen/Library/CloudStorage/OneDrive-Personal/Dokumenter/AAU Datavidenskab/6. Semester/P6 Project/EV scheduling/acndata_sessions.json")
EVdf = pd.DataFrame(EV["_items"])
EVdf

,_id,clusterID,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
0,5bc90cb9f9af8b0d7fe77cd2,0039,"Wed, 25 Apr 2018 11:08:04 GMT","Wed, 25 Apr 2018 13:20:10 GMT","Wed, 25 Apr 2018 13:21:10 GMT",7.932,2_39_78_362_2018-04-25 11:08:04.400812,0002,CA-496,2-39-78-362,America/Los_Angeles,None,None
1,5bc90cb9f9af8b0d7fe77cd3,0039,"Wed, 25 Apr 2018 13:45:10 GMT","Thu, 26 Apr 2018 00:56:16 GMT","Wed, 25 Apr 2018 16:44:15 GMT",10.013,2_39_95_27_2018-04-25 13:45:09.617470,0002,CA-319,2-39-95-27,America/Los_Angeles,None,None
2,5bc90cb9f9af8b0d7fe77cd4,0039,"Wed, 25 Apr 2018 13:45:50 GMT","Wed, 25 Apr 2018 23:04:45 GMT","Wed, 25 Apr 2018 14:51:44 GMT",5.257,2_39_79_380_2018-04-25 13:45:49.962001,0002,CA-489,2-39-79-380,America/Los_Angeles,None,None
3,5bc90cb9f9af8b0d7fe77cd5,0039,"Wed, 25 Apr 2018 14:37:06 GMT","Wed, 25 Apr 2018 23:55:34 GMT","Wed, 25 Apr 2018 16:05:22 GMT",5.177,2_39_79_379_2018-04-25 14:37:06.460772,0002,CA-327,2-39-79-379,America/Los_Angeles,None,None
4,5bc90cb9f9af8b0d7fe77cd6,0039,"Wed, 25 Apr 2018 14:40:34 GMT","Wed, 25 Apr 2018 23:03:12 GMT","Wed, 25 Apr 2018 17:40:30 GMT",10.119,2_39_79_381_2018-04-25 14:40:33.638896,0002,CA-490,2-39-79-381,America/Los_Angeles,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31419,61550519f9af8b76960e169a,0039,"Mon, 13 Sep 2021 21:12:53 GMT","Tue, 14 Sep 2021 00:25:36 GMT","Mon, 13 Sep 2021 21:41:31 GMT",2.285,2_39_89_25_2021-09-13 21:12:53.318460,0002,CA-315,2-39-89-25,America/Los_Angeles,000000431,"[{'WhPerMile': 286, 'kWhRequested': 28.6, 'mil..."
31420,61550519f9af8b76960e169b,0039,"Mon, 13 Sep 2021 21:17:04 GMT","Tue, 14 Sep 2021 01:01:49 GMT","Mon, 13 Sep 2021 23:18:07 GMT",6.715,2_39_123_23_2021-09-13 21:16:44.026068,0002,CA-313,2-39-123-23,America/Los_Angeles,000006481,"[{'WhPerMile': 460, 'kWhRequested': 11.5, 'mil..."
31421,61550519f9af8b76960e169c,0039,"Mon, 13 Sep 2021 22:33:07 GMT","Mon, 13 Sep 2021 23:06:55 GMT",None,17.720,2_39_81_4550_2021-09-13 22:33:04.543952,0002,11900388,2-39-81-4550,America/Los_Angeles,000003905,"[{'WhPerMile': 300, 'kWhRequested': 30.0, 'mil..."
31422,61550519f9af8b76960e169d,0039,"Mon, 13 Sep 2021 23:11:12 GMT","Tue, 14 Sep 2021 01:43:11 GMT","Tue, 14 Sep 2021 00:13:35 GMT",2.018,2_39_91_437_2021-09-13 23:10:59.528292,0002,CA-317,2-39-91-437,America/Los_Angeles,000001082,"[{'WhPerMile': 290, 'kWhRequested': 5.8, 'mile..."


In [24]:
summary = EVdf.describe(include="all")

In [26]:
summary.T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
_id,31424,31424,5bc90cb9f9af8b0d7fe77cd2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
clusterID,31424,2,0039,29343,NaN,NaN,NaN,NaN,NaN,NaN,NaN
connectionTime,31424,31380,"Tue, 28 Aug 2018 15:54:35 GMT",8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
disconnectTime,31424,31386,"Wed, 08 Aug 2018 02:32:45 GMT",5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
doneChargingTime,29369,29253,"Tue, 16 Oct 2018 16:40:56 GMT",8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
kWhDelivered,31424.0,NaN,NaN,NaN,9.253283,8.760917,0.501,3.402,6.716,12.94475,77.7
sessionID,31424,31424,2_39_78_362_2018-04-25 11:08:04.400812,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
siteID,31424,2,0002,29343,NaN,NaN,NaN,NaN,NaN,NaN,NaN
spaceID,31424,55,CA-303,1808,NaN,NaN,NaN,NaN,NaN,NaN,NaN
stationID,31424,55,2-39-139-28,1808,NaN,NaN,NaN,NaN,NaN,NaN,NaN
